In [1]:
# pip install -r requirements.txt

In [2]:
from pandas import read_csv

In [3]:
x = read_csv("MOS Data.csv")
z = x.sample(n=10)
z

,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet)
11,12,49,11,678,105
13,14,53,12,670,115
23,24,73,17,612,165
24,25,75,17,605,170
29,30,85,20,570,195
0,1,27,5,500,50
10,11,47,10,682,100
7,8,41,9,680,85
9,10,45,10,686,95
18,19,63,14,647,140


In [4]:
n = list(z.iloc[:,1])
f = n[0]
x = []
for i in n:
    x.append( i/f)
z["Modulus Ratio (N)"] = x
z

,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet),Modulus Ratio (N)
11,12,49,11,678,105,1.000000
13,14,53,12,670,115,1.081633
23,24,73,17,612,165,1.489796
24,25,75,17,605,170,1.530612
29,30,85,20,570,195,1.734694
0,1,27,5,500,50,0.551020
10,11,47,10,682,100,0.959184
7,8,41,9,680,85,0.836735
9,10,45,10,686,95,0.918367
18,19,63,14,647,140,1.285714


In [5]:
h = [0.04]*10
z["Height"] = h
b = [0.2] * 10
z["Breadth"] = b

eq = [i*0.2 for i in x]
z["Equivalent Breadth"] = eq

eqar = [i*0.04 for i in eq]
z["Equivalent Area"] = eqar

y = [ 0.38 - 0.04*i for i in range(0,10,1)]
z["y"] = y

yar = [ eqar[i] * y[i] for i in range(len(x))]
sum_yar = sum(yar)
sum_ar = sum(eqar)
yc = sum_yar/sum_ar
print("yc = ",yc)
z

yc =  0.20831541218637992


,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet),Modulus Ratio (N),Height,Breadth,Equivalent Breadth,Equivalent Area,y
11,12,49,11,678,105,1.000000,0.04,0.2,0.200000,0.008000,0.38
13,14,53,12,670,115,1.081633,0.04,0.2,0.216327,0.008653,0.34
23,24,73,17,612,165,1.489796,0.04,0.2,0.297959,0.011918,0.30
24,25,75,17,605,170,1.530612,0.04,0.2,0.306122,0.012245,0.26
29,30,85,20,570,195,1.734694,0.04,0.2,0.346939,0.013878,0.22
0,1,27,5,500,50,0.551020,0.04,0.2,0.110204,0.004408,0.18
10,11,47,10,682,100,0.959184,0.04,0.2,0.191837,0.007673,0.14
7,8,41,9,680,85,0.836735,0.04,0.2,0.167347,0.006694,0.10
9,10,45,10,686,95,0.918367,0.04,0.2,0.183673,0.007347,0.06
18,19,63,14,647,140,1.285714,0.04,0.2,0.257143,0.010286,0.02


In [6]:
yi = [i-yc for i in y]
z["yi"] = yi
z

,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet),Modulus Ratio (N),Height,Breadth,Equivalent Breadth,Equivalent Area,y,yi
11,12,49,11,678,105,1.000000,0.04,0.2,0.200000,0.008000,0.38,0.171685
13,14,53,12,670,115,1.081633,0.04,0.2,0.216327,0.008653,0.34,0.131685
23,24,73,17,612,165,1.489796,0.04,0.2,0.297959,0.011918,0.30,0.091685
24,25,75,17,605,170,1.530612,0.04,0.2,0.306122,0.012245,0.26,0.051685
29,30,85,20,570,195,1.734694,0.04,0.2,0.346939,0.013878,0.22,0.011685
0,1,27,5,500,50,0.551020,0.04,0.2,0.110204,0.004408,0.18,-0.028315
10,11,47,10,682,100,0.959184,0.04,0.2,0.191837,0.007673,0.14,-0.068315
7,8,41,9,680,85,0.836735,0.04,0.2,0.167347,0.006694,0.10,-0.108315
9,10,45,10,686,95,0.918367,0.04,0.2,0.183673,0.007347,0.06,-0.148315
18,19,63,14,647,140,1.285714,0.04,0.2,0.257143,0.010286,0.02,-0.188315


In [7]:
I = [eq[i]*pow(0.04,3)/12 + eq[i]*0.04*pow(yi[i],2) for i in range(10)]
z["I"] = I
z

,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet),Modulus Ratio (N),Height,Breadth,Equivalent Breadth,Equivalent Area,y,yi,I
11,12,49,11,678,105,1.000000,0.04,0.2,0.200000,0.008000,0.38,0.171685,0.000237
13,14,53,12,670,115,1.081633,0.04,0.2,0.216327,0.008653,0.34,0.131685,0.000151
23,24,73,17,612,165,1.489796,0.04,0.2,0.297959,0.011918,0.30,0.091685,0.000102
24,25,75,17,605,170,1.530612,0.04,0.2,0.306122,0.012245,0.26,0.051685,0.000034
29,30,85,20,570,195,1.734694,0.04,0.2,0.346939,0.013878,0.22,0.011685,0.000004
0,1,27,5,500,50,0.551020,0.04,0.2,0.110204,0.004408,0.18,-0.028315,0.000004
10,11,47,10,682,100,0.959184,0.04,0.2,0.191837,0.007673,0.14,-0.068315,0.000037
7,8,41,9,680,85,0.836735,0.04,0.2,0.167347,0.006694,0.10,-0.108315,0.000079
9,10,45,10,686,95,0.918367,0.04,0.2,0.183673,0.007347,0.06,-0.148315,0.000163
18,19,63,14,647,140,1.285714,0.04,0.2,0.257143,0.010286,0.02,-0.188315,0.000366
